## 1. 다운로드 및 데이터 전처리

### Settings

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
  pass

Importing the required libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
keras = tf.keras
import tensorflow_datasets as tfds
tf.__version__

Check out the details (features, statistics, etc.) of the dataset to be downloaded here: https://www.tensorflow.org/datasets/datasets

### 플라워 데이터셋 다운로드 및 로드하기 `tfds.load()`

In [ ]:
# Since "tf_flowers" doesn't define standard splits, 
# use the subsplit feature to divide it into (train, validation, test)
# with 80%, 10%, 10% of the data respectively.

(raw_train, raw_validation, raw_test), metadata = tfds.load(name="tf_flowers", 
                                                            with_info=True,
                                                            split=('train[:80%]', 'train[80%:90%]', 'train[90%:]'),
# specifying batch_size=-1 will load full dataset in the memory
#                                                             batch_size=-1,
# as_supervised: `bool`, if `True`, the returned `tf.data.Dataset`
# will have a 2-tuple structure `(input, label)`                                                            
                                                            as_supervised=True)

In [ ]:
print(raw_train)
print(raw_validation)
print(raw_test)

데이터 파이프라인을 통한 전처리 함수를 정의합니다.

In [ ]:
# 기본적인 데이터 전처리 수행하기

IMG_SIZE = 128
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

def format_example(image, label):
    print("Format example called!")
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    # Resize the image if required
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

### 효율적으로 데이터 증폭시키기
Read more [here](https://stackoverflow.com/questions/55141076/how-to-apply-data-augmentation-in-tensorflow-2-0-after-tfds-load) to know how the map function applies data augmentation

In [ ]:
def augment(image,label):
  # Add 6 pixels of padding
  image = tf.image.resize_with_crop_or_pad(image, IMG_SIZE + 6, IMG_SIZE + 6) 
   # Random crop back to the original size
  image = tf.image.random_crop(image, size=[IMG_SIZE, IMG_SIZE, 3])
  image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness
  image = tf.clip_by_value(image, 0, 1)
  return image, label

train = train.map(augment)

또한 우리는 데이터셋을 섞어 샘플 순서에 따른 편향을 없앱니다. 또한 32 크기의 미니배치를 생성하고, 데이터셋이 prefetch 될 수 있도록 설정합니다.

Without prefetch, the CPU and the GPU/TPU sit idle much of the time

![alt text](https://www.tensorflow.org/images/datasets_without_pipelining.png)

With prefetch, idle time diminishes significantly

![alt text](https://www.tensorflow.org/images/datasets_with_pipelining.png)

In [ ]:
SHUFFLE_BUFFER_SIZE = 1024
BATCH_SIZE = 32

train = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation = validation.batch(BATCH_SIZE)
test = test.batch(BATCH_SIZE)
# (Optional) prefetch will enable the input pipeline to asynchronously fetch batches while
# your model is training.
train = train.prefetch(tf.data.experimental.AUTOTUNE)
print(train)
print(validation)
print(test)

### 데이터 시각화하기

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Get the function which converts label indices to string
get_label_name = metadata.features['label'].int2str

Here we fetch the datatset batch by batch and convert it to numpy array just before passing it to plotting function.

In [ ]:
plt.figure(figsize=(12,12)) 
for batch in train.take(1):
    for i in range(9):
        image, label = batch[0][i], batch[1][i]
        plt.subplot(3,3,i+1)
        plt.imshow(image.numpy())
        plt.title(get_label_name(label.numpy()))
        plt.grid(False)
print(image.shape)
print(np.min(image))
print(np.max(image))

## 2.  `tf.keras`로 모델 빌드하기

In [ ]:
from tensorflow import keras
keras.__version__

In [ ]:
# Creating a simple CNN model in keras using functional API
def create_model():
    img_inputs = keras.Input(shape=IMG_SHAPE)
    conv_1 = keras.layers.Conv2D(32, (3, 3), activation='relu')(img_inputs)
    maxpool_1 = keras.layers.MaxPooling2D((2, 2))(conv_1)
    conv_2 = keras.layers.Conv2D(64, (3, 3), activation='relu')(maxpool_1)
    maxpool_2 = keras.layers.MaxPooling2D((2, 2))(conv_2)
    conv_3 = keras.layers.Conv2D(64, (3, 3), activation='relu')(maxpool_2)
    flatten = keras.layers.Flatten()(conv_3)
    dense_1 = keras.layers.Dense(64, activation='relu')(flatten)
    output = keras.layers.Dense(metadata.features['label'].num_classes, activation='softmax')(dense_1)

    model = keras.Model(inputs=img_inputs, outputs=output)
    
    return model

### Visualizing the model

summary를 통해 model의 정보를 요약하고, plot_model 메서드로 모델의 구조를 시각화할 수 있습니다.

In [ ]:
simple_model = create_model()
simple_model.summary()

In [ ]:
keras.utils.plot_model(simple_model, 'flower_model_with_shape_info.png', show_shapes=True)

### Setting training parameters

In [ ]:
num_train, num_val, num_test = (
  metadata.splits['train'].num_examples * weight/10 for weight in [8,1,1]
)

steps_per_epoch = round(num_train)//BATCH_SIZE
validation_steps = round(num_val)//BATCH_SIZE

print('Number of examples in the train set:', num_train)
print('Number of examples in the validation set:', num_val)
print('Number of examples in the test set:', num_test)

### Compile and train the model

In [ ]:
import os

def train_model(model):
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Creating Keras callbacks 
    os.makedirs('training_checkpoints/', exist_ok=True)
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
        'training_checkpoints/weights.{epoch:02d}-{val_loss:.2f}.h5')
    early_stopping_checkpoint = keras.callbacks.EarlyStopping(patience=5)

    history = model.fit(train,
              epochs=50, 
              validation_data=validation,
              callbacks=[model_checkpoint_callback,
                         early_stopping_checkpoint])
    
    return history

In [ ]:
history = train_model(simple_model)

Plotting the training and validation metrics returned by the `train_model()` routine. We use matplotlib to plot the graph.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

그래프는 모델 학습이 어떻게 이루어졌는지 알려줍니다. 학습과 검증데이터에 대해 정확도는 상승하고, 비용은 감시했는지 확인해야합니다.

만약 학습 정확도는 높지만 검증 정확도가 낮다면, 전형적인 과대적합의 케이스입니다. 이런 경우 학습데이터를 더 늘리거나 데이터증폭이 필요할 수 있습니다. 또한 Batchnorm이나 Dropout과 같은 정규화 기능을 갖는 모델을 시도해볼 수 있습니다.

## 3. Using pre-trained model

pre-trained model 사용의 개념은 구글, 아마존 그리고 MS와 같은 기업에서 개발된 모델을 사용하는 것입니다. 그런 모델들은 1000개의 카테고리로 구성된 140만개의 이미지인 ImageNet 데이터셋을 학습했습니다. 이런 모델은 1000개의 오브젝트에 대한 기본적인 특징을 이미 학습했기 때문에 강력한 특징추출 능력을 갖고 있습니다.

In [ ]:
from tensorflow import keras

# Create the base model from the pre-trained model MobileNet V2
base_model = keras.applications.InceptionV3(input_shape=IMG_SHAPE,
# We cannot use the top classification layer of the pre-trained model as it contains 1000 classes.
# It also restricts our input dimensions to that which this model is trained on (default: 299x299)
                                               include_top=False, 
                                               weights='imagenet')

In [ ]:
base_model.trainable = False
# Let's take a look at the base model architecture
base_model.summary()

In [ ]:
keras.utils.plot_model(base_model, 'inception_model_with_shape_info.png', show_shapes=True)

우리가 가져온 네트워크는 128x128x3 사진을 영상특징들로 변환하는 특징추출기(feature extractor)입니다. include_top=False 옵션을 통해, 우리 데이터를 위한 분류기를 추가해 학습할 수 있습니다. 

NOTE: It's important to freeze the convolutional based before we compile and train the model. By freezing (or setting `layer.trainable = False`), we prevent the weights in these layers from being updated during training.

### Adding classification head

예측을 생성하기 위해 특징들의 spatial average를 계산하기 위해 `tf.keras.layers.GlobalAveragePooling2d` 레이어를 사용합니다. GAP 레이어는 특징맵을 벡터로 변환해주고, 그 뒤 FCN을 연결해 분류합니다. 

In [ ]:
def build_model():
    
    model = keras.Sequential([
        base_model,
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(metadata.features['label'].num_classes, 
                           activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

inception_model = build_model()
inception_model.summary()

### Train the InceptionV3 model

In [ ]:
# Evaluating model before training (Optional)
loss0, accuracy0 = inception_model.evaluate(validation)

In [ ]:
# Creating Keras callbacks 
os.makedirs('training_checkpoints/', exist_ok=True)
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
        'training_checkpoints/weights.{epoch:02d}-{val_loss:.2f}.h5')
early_stopping_checkpoint = keras.callbacks.EarlyStopping(patience=5)

history = inception_model.fit(train,
                              epochs=5,
                              validation_data=validation, 
                              callbacks=[model_checkpoint_callback,
                              early_stopping_checkpoint])

After training the model for 5 epochs we were able to get ~70% accuracy. We plot the learning curves of the training and validation accuracy / loss.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
inception_model.evaluate(test)

### Fine tuning the model

In [ ]:
# Un-freeze the top layers of the model
base_model.trainable = True
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

In [ ]:
# Fine tune from this layer onwards
fine_tune_at = 249

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False
    
# Compile the model using a much-lower training rate.
inception_model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
inception_model.summary()

If you trained to convergence earlier, this will get you a few percent more accuracy.

In [ ]:
history_fine = inception_model.fit(train, 
                                  epochs=10, 
                                  initial_epoch = 5,
                                  validation_data=validation, 
                                  callbacks=[model_checkpoint_callback,
                                  early_stopping_checkpoint])

Note: If the training dataset is fairly small, and is similar to the original datasets that Inception V3 was trained on, so fine-tuning may result in overfitting.

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

initial_epochs=5

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.plot([initial_epochs-1,initial_epochs-1], 
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.plot([initial_epochs-1,initial_epochs-1], 
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
inception_model.evaluate(test)


In summary here is what we covered on how to do transfer learning using a pre-trained model to improve accuracy: 
* Using a pre-trained model for <b>feature extraction</b> - when working with a small dataset, it is common to leverage the features learned by a model trained on a larger dataset in the same domain. This is done by instantiating the pre-trained model and adding a fully connected classifier on top. The pre-trained model is "frozen" and only the weights of the classifier are updated during training. In this case, the convolutional base extracts all the features associated with each image and we train a classifier that determines, given these set of features to which class it belongs. 
* <b>Fine-tuning</b> a pre-trained model - to further improve performance, one might want to repurpose the top-level layers of the pre-trained models to the new dataset via fine-tuning. In this case, we tune our weights such that we learn highly specified and high level features specific to our dataset. This only make sense when the training dataset is large and very similar to the orginial dataset that the pre-trained model was trained on.

References: 
1. https://www.tensorflow.org/alpha/tutorials/images/intro_to_cnns
2. https://www.tensorflow.org/alpha/tutorials/images/transfer_learning